In [1]:
import matplotlib.pyplot as plt
import plotly.express as px
import os
import pandas as pd
from dataset.ipc2023_learning_domain_info import IPC2023_LEARNING_DOMAINS, DIFFICULTIES, PROBLEM_INDICES
from itertools import product

In [2]:
save_dir = "px/uncertainty"
os.makedirs(save_dir, exist_ok=True)

In [3]:
diff_to_int = {
    "easy": 0,
    "medium": 1,
    "hard": 2,
}

KEYS = {"domain", "ratio", "std", "h", "prob"}

stats = {k:[] for k in KEYS}
for domain in IPC2023_LEARNING_DOMAINS:
    for difficulty, problem in product(DIFFICULTIES, PROBLEM_INDICES):
        f = f"icaps24_gp_correlation_logs/{domain}_{difficulty}_{problem}_gp.log"
        if not os.path.exists(f):
            continue

        with open(f, "r") as file:
            start = False
            for line in file.readlines():
                if "Number of unseen" in line:
                    start = True
                    continue

                if start:
                    toks = line.split()
                    if len(toks) != 3:
                        break
                    
                    stats["domain"].append(domain)
                    stats["ratio"].append(float(toks[0]))
                    stats["std"].append(float(toks[1]))
                    stats["h"].append(float(toks[2]))
                    stats["prob"].append(float(diff_to_int[difficulty] * 30 + int(problem[1:])))

df = pd.DataFrame(data=stats)

In [4]:
fig = px.scatter(df, "ratio", "std", color="domain", log_y=True, range_y=(1, 1e3))
fig.write_html(f"{save_dir}/std_vs_ratio.html")

In [5]:
fig = px.scatter(df, "prob", "std", color="domain", log_y=True, range_y=(1, 1e3))
fig.write_html(f"{save_dir}/std_vs_prob.html")

In [6]:
fig = px.scatter(df, "prob", "ratio", color="domain", range_y=(0, 1))
fig.write_html(f"{save_dir}/ratio_vs_prob.html")

In [7]:
fig = px.scatter(df, "h", "std", color="domain", log_x=True, log_y=True, range_y=(1, 1e3))
fig.write_html(f"{save_dir}/std_vs_h.html")

In [8]:
fig = px.scatter(df, "h", "ratio", color="domain", log_x=True, range_y=(0, 1))
fig.write_html(f"{save_dir}/ratio_vs_h.html")